### 1.Run commands needed to run code:

Used models: <br>
https://huggingface.co/flax-community/t5-recipe-generation <br>
https://huggingface.co/Helsinki-NLP/opus-mt-ru-en<br>
https://huggingface.co/SnypzZz/Llama2-13b-Language-translate<br>

In [ ]:
pip install transformers  

In [ ]:
pip install --upgrade jax jaxlib # CPU-only  

In [ ]:
pip install flax       

In [ ]:
pip install torch torchvision torchaudio


In [ ]:
pip install sentencepiece

In [ ]:
pip install sacremoses

#### 2. Declare your input and output language

In [13]:
language_cd = "en"

In [2]:
language_cd = "pl"

#### 3. Declare all your available grocery items

In [15]:
items = [
    "macaroni, butter, salt, bacon, milk, flour, pepper, cream corn",
    "provolone cheese, bacon, bread, ginger",
    "pepper"
]

In [3]:
items = [
    "wieprzowe mięso mielone, pieczarki, pieprz, sól, majeranek, tymianek, bazylia, papryka słodka, oliwa, olej, makaron, mąka, jajka"
]

#### 5. Load model for translation

In [4]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast,MarianMTModel, MarianTokenizer


def translate_polish_to_english(text):
    model_name = "Helsinki-NLP/opus-mt-pl-en"
    model = MarianMTModel.from_pretrained(model_name)
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    translation = model.generate(**inputs)
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
    return translated_text


def translate_english_to_polish(text):
    model = MBartForConditionalGeneration.from_pretrained("SnypzZz/Llama2-13b-Language-translate")
    tokenizer_eng = MBart50TokenizerFast.from_pretrained("SnypzZz/Llama2-13b-Language-translate", src_lang="en_XX")
    model_inputs = tokenizer_eng(text, return_tensors="pt")
    generated_tokens = model.generate(
        **model_inputs,
        forced_bos_token_id=tokenizer_eng.lang_code_to_id["pl_PL"]
    )
    english_to_polish = tokenizer_eng.batch_decode(generated_tokens, skip_special_tokens=True)
    return english_to_polish

In [5]:
if language_cd == 'pl':
    for el in items:
        translated_items = translate_polish_to_english(el)
        items.append(translated_items)
        items.remove(el)
print(items)

['pork minced meat, mushrooms, pepper, salt, marjoram, thyme, basil, sweet peppers, oil, pasta, flour, eggs']


### 6. Load chef model

In [6]:
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

prefix = "items: "
generation_kwargs = {
    "max_length": 512,
    "min_length": 64,
    "no_repeat_ngram_size": 3,
    "do_sample": True,
    "top_k": 60,
    "top_p": 0.95
}

special_tokens = tokenizer.all_special_tokens
tokens_map = {
    "<sep>": "--",
    "<section>": "\n"
}


def skip_special_tokens(text, special_tokens):
    for token in special_tokens:
        text = text.replace(token, "")

    return text


def target_postprocessing(texts, special_tokens):
    if not isinstance(texts, list):
        texts = [texts]

    new_texts = []
    for text in texts:
        text = skip_special_tokens(text, special_tokens)

        for k, v in tokens_map.items():
            text = text.replace(k, v)

        new_texts.append(text)

    return new_texts


def generation_function(texts):
    _inputs = texts if isinstance(texts, list) else [texts]
    inputs = [prefix + inp for inp in _inputs]
    inputs = tokenizer(
        inputs,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="jax"
    )

    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        **generation_kwargs
    )
    generated = output_ids.sequences
    generated_recipe = target_postprocessing(
        tokenizer.batch_decode(generated, skip_special_tokens=False),
        special_tokens
    )
    return generated_recipe

generated = generation_function(items)
text = generated[0].split("\n")
final_translated_receipe = []
if language_cd == 'pl':
    for i in text:
        final_translated_receipe.append(translate_english_to_polish(i))
else:
    for i in text:
        final_translated_receipe.append(i)
for section in final_translated_receipe:
    steps = section[0].split("--")
    for el in steps:
        print(el)

Tytuł: minced stroganoff
Składniki: 400 g mięsa wieprzowego mielonego
 10 g grzybów, nakrojonych
 1/8 łyżeczka pieprzu
 1/8 łyżeczka soli
 1/2 łyżeczka marjoram
 1/2 łyżeczka tymiany
 1/4 łyżeczka bazyliki
 4 słodkie pieprzy, przekrojone i drobnie nakrojone
 olej, do smażenia
 makarony
 250 g mąki ogólnej
 2 jaja
kierunki: ogrzewanie oleju w patelni do smażenia.
 dodać grzyby i smażyć na minutę lub tak.
 dodać mięso i smażyć na 5 minut na średnim ogniu.
 sezon z przyprawami i dodać pieprzy.
 mieszać smażyć na kolejne 3 minuty.
 dodać makaron do patelni i dobrze mieszać.
 pokryć patelni i gotować na 10 minut lub dopóki makaron jest gotowany.
 do servirowania, łyżka mielony stroganoff nad makaronem.
